# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。
回忆一下在 :numref:`sec_model_construction`对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。


In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

让我们向该层提供一些数据，验证它是否能按预期工作。


In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以[**将层作为组件合并到更复杂的模型中**]。


In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.1223e-09, grad_fn=<MeanBackward0>)

## [**带参数的层**]

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。


In [8]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
net = MyLinear(4, 1)
net.weight.requires_grad, net.weight.requires_grad_

(True, <function Parameter.requires_grad_>)

接下来，我们实例化`MyLinear`类并访问其模型参数。


In [9]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.9262,  1.1798, -2.2143],
        [ 1.7689, -0.0235, -0.9656],
        [ 0.3159,  0.5352,  0.5828],
        [ 1.1320,  0.3567,  0.7671],
        [ 0.5626,  2.2917,  1.0367]], requires_grad=True)

我们可以[**使用自定义层直接执行前向传播计算**]。


In [10]:
linear(torch.rand(2, 5))

tensor([[2.8173, 2.6411, 0.0000],
        [3.4071, 2.0153, 0.0000]])

我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。


In [11]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.4859],
        [1.8184]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个返回输入数据的傅立叶系数前半部分的层。


In [27]:
from typing import List
import torch
from torch import nn

class Layer(nn.Module):

    def __init__(self, h_w: List[int], k: int) -> None:
        super().__init__()
        h, w = h_w
        self.weight = nn.Parameter(torch.Tensor(k, w, h))
        self.initialized = False

    def forward(self, X):
        if not self.initialized:
            nn.init.xavier_uniform_(self.weight)
            self.initialized = True
        y = torch.einsum('bwh, hw->b', [self.weight, X])
        return y

net = Layer([5, 7], 3)

X = torch.rand(5, 7)
output = net(X)
output


tensor([-0.2931, -1.4343,  0.3365], grad_fn=<ViewBackward0>)

In [44]:
import torch
import torch.nn as nn
import torch.fft

class FourierCoeffLayer(nn.Module):

    def forward(self, x):
        # 对输入数据进行傅立叶变换
        x_complex = torch.view_as_complex(x)
        x_fft = torch.fft.fft(x_complex)
        # 取前半部分的傅立叶系数
        n = x.size(-2) // 2
        x_fft_half = x_fft[..., :n]
        
        return x_fft_half

# 使用示例
input_data = torch.rand(1, 4, 2)  # 假设输入是一批大小为64的数据
fourier_layer = FourierCoeffLayer()
output = fourier_layer(input_data)

# output 包含了输入数据的傅立叶系数前半部分
output

tensor([[ 2.0364+1.6014j, -0.4392-0.6163j]])

In [45]:
x = torch.rand(1, 4, 2)  # 一个实数张量
x_complex = torch.view_as_complex(x)  # 转换为复数张量
x_fft = torch.fft.fft(x_complex)  # 执行傅立叶变换
x_fft, x_complex, x

(tensor([[ 2.3282+1.3852j,  1.1607-0.8584j,  0.0495-0.2110j, -0.3857+0.1931j]]),
 tensor([[0.7882+0.1272j, 0.8326+0.7857j, 0.4007+0.4599j, 0.3068+0.0125j]]),
 tensor([[[0.7882, 0.1272],
          [0.8326, 0.7857],
          [0.4007, 0.4599],
          [0.3068, 0.0125]]]))

[Discussions](https://discuss.d2l.ai/t/1835)
